In [84]:
import pandas as pd
from scipy.optimize import linprog
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np


In [85]:
df = pd.read_csv(r'DEA.csv')





In [86]:
input_cols_300 = [
    'Mean Energy Consumption (300 Output Tokens)',
    'PUE',
    'WUE (Site)',
    "WUE (Source)",
    'CIF'
]
input_cols_1000 = [
    'Mean Energy Consumption (1000 Output Tokens)',
    'PUE',
    'WUE (Site)',
    "WUE (Source)",    'CIF'
]
input_cols_1500 = [
    'Mean Energy Consumption (1500 Output Tokens)',
    'PUE',
    'WUE (Site)',
    "WUE (Source)",    'CIF'
]
output_cols = [
    'Score',

]

In [87]:

df.head()

,Model,Organization,API Provider,GPUs Used,System (DGX/HGX),Critical Power (kW),Idle Power,Utilization Rate (min),Utilization Rate (max),PUE,...,Mean Energy Consumption (300 Output Tokens),Mean Energy Consumption (1000 Output Tokens),Mean Energy Consumption (1500 Output Tokens),Score,Min Mean (300 Output Tokens),Max Mean (300 Output Tokens),Min Mean (1k Output Tokens),Max Mean (1k Output Tokens),Min Mean (1.5k Output Tokens),Max Mean (1.5k Output Tokens)
0,GPT-4.1,OpenAI,OpenAI,Nvidia H200/H100,HGX H200/H100,10.2,1.84,20.00%,40.00%,1.12,...,0.000918,0.002513,0.004233,53,0.000841,0.000995,0.002302,0.002724,0.003877,0.004588
1,GPT-4.1 mini,OpenAI,OpenAI,Nvidia H200/H100,HGX H200/H100,10.2,1.84,10.00%,20.00%,1.12,...,0.000421,0.001680,0.001590,53,0.000371,0.000470,0.000748,0.000947,0.001404,0.001777
2,GPT-4.1 nano,OpenAI,OpenAI,Nvidia H200/H100,HGX H200/H100,10.2,1.84,10.00%,20.00%,1.12,...,0.000103,0.000559,0.000454,41,0.000091,0.000115,0.000239,0.000303,0.000401,0.000507
3,o4-mini (high),OpenAI,OpenAI,Nvidia H200/H100,HGX H200/H100,10.2,1.84,10.00%,20.00%,1.12,...,0.002916,0.002655,0.005666,70,0.002574,0.003259,0.004447,0.005631,0.005000,0.006331
4,GPT-4.5,OpenAI,OpenAI,Nvidia H200/H100,HGX H200/H100,10.2,1.84,20.00%,40.00%,1.12,...,0.006723,0.012769,0.030495,51,0.006158,0.007287,0.018779,0.022220,0.027935,0.033055


In [88]:
df.isna().sum()

Model                                           0
Organization                                    0
API Provider                                    0
GPUs Used                                       0
System (DGX/HGX)                                0
Critical Power (kW)                             0
Idle Power                                      0
Utilization Rate (min)                          0
Utilization Rate (max)                          0
PUE                                             0
WUE (Site)                                      0
WUE (Source)                                    0
CIF                                             0
Power Consumption per system (Min)(kW)          0
Power Consumption per system (Max) (kW)         0
Mean Energy Consumption (300 Output Tokens)     0
Mean Energy Consumption (1000 Output Tokens)    0
Mean Energy Consumption (1500 Output Tokens)    2
Score                                           0
Min Mean (300 Output Tokens)                    0


In [89]:
df.loc[df['Mean Energy Consumption (1500 Output Tokens)'] == '#DIV/0!', 'Mean Energy Consumption (1500 Output Tokens)'] = 0.0

In [90]:
len(df[input_cols_1000].dropna())

27

In [91]:

def normalize_matrix(matrix):
    scaler = MinMaxScaler()
    return scaler.fit_transform(matrix)



X_300 = df[input_cols_300].astype(float).values 
X_1000 = df[input_cols_1000].astype(float).values  
X_1500 = df[input_cols_1500].dropna().astype(float).values  
X_300 = normalize_matrix(X_300)
X_1000 = normalize_matrix(X_1000)
X_1500 = normalize_matrix(X_1500)


Y = df[output_cols].astype(float).values  
Y = normalize_matrix(Y)

Y_1500 = df.dropna(subset=['Mean Energy Consumption (1500 Output Tokens)'])[output_cols].astype(float).values  
n_dmus, n_inputs = X_300.shape

n_dmus_1500, n_inputs_1500 = X_1500.shape




_, n_outputs = Y.shape

eff_scores = []





In [92]:
for i in input_cols_300:
    for j in output_cols:
        print(f'{i},{j}":", {df[i].corr(df[j])}')

Mean Energy Consumption (300 Output Tokens),Score":", 0.3975782063738578
PUE,Score":", 0.02542688762373754
WUE (Site),Score":", 0.3520112490154502
WUE (Source),Score":", 0.20530996531204385
CIF,Score":", 0.029756105606774896


In [93]:
X_dict = {
    '300': X_300,
    '1000': X_1000,
}

In [96]:
for i in X_dict:


    u = np.zeros(n_dmus)             
    v = np.zeros((n_dmus, n_inputs)) 

    for k in range(n_dmus):
 
        c = [-Y.flatten()[k]] + [0.0]*n_inputs


        A_ub, b_ub = [], []
        for j in range(n_dmus):
            A_ub.append([ Y.flatten()[j] ] + list(-X_dict[i][j]))
            b_ub.append(0.0)

        A_eq = [[0.0] + list(X_dict[i][k])]
        b_eq = [1.0]

        eps = 1e-6
        bounds = [(eps, None)] * (1 + n_inputs)

        res = linprog(
            c=c,
            A_ub=A_ub, b_ub=b_ub,
            A_eq=A_eq, b_eq=b_eq,
            bounds=bounds,
            method='highs'
        )

        if not res.success:
            raise RuntimeError(f"DEA multiplier LP failed for DMU {k}")

        u[k]        = res.x[0]
        v[k, :]     = res.x[1:]

    CE = np.zeros((n_dmus, n_dmus))
    for k in range(n_dmus):
        for j in range(n_dmus):
            CE[k, j] = (u[k] * Y[j]) / (v[k].dot(X_dict[i][j]))

    avg_CE = CE.mean(axis=0)   

    # 4) Attach back to your DataFrame
    df[f'Cross_Efficiency_Avg_{i}'] = avg_CE



/var/folders/k0/5kjb5xhn5hz5qd2_5ccp61th0000gn/T/ipykernel_50157/1131832512.py:41: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

/var/folders/k0/5kjb5xhn5hz5qd2_5ccp61th0000gn/T/ipykernel_50157/1131832512.py:41: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



In [97]:



u = np.zeros(n_dmus_1500)             
v = np.zeros((n_dmus_1500, n_inputs_1500))

for k in range(n_dmus_1500):

        c = [-Y_1500.flatten()[k]] + [0.0]*n_inputs_1500


        A_ub, b_ub = [], []
        for j in range(n_dmus_1500):
            A_ub.append([ Y_1500.flatten()[j] ] + list(-X_1500[j]))
            b_ub.append(0.0)

        A_eq = [[0.0] + list(X_1500[k])]
        b_eq = [1.0]

        eps = 1e-6
        bounds = [(eps, None)] * (1 + n_inputs_1500)

        res = linprog(
            c=c,
            A_ub=A_ub, b_ub=b_ub,
            A_eq=A_eq, b_eq=b_eq,
            bounds=bounds,
            method='highs'
        )

        if not res.success:
            raise RuntimeError(f"DEA multiplier LP failed for DMU {k}")

        u[k]        = res.x[0]
        v[k, :]     = res.x[1:]

CE = np.zeros((n_dmus_1500, n_dmus_1500))
for k in range(n_dmus_1500):
        for j in range(n_dmus_1500):
            CE[k, j] = (u[k] * Y_1500[j]) / (v[k].dot(X_1500[j]))

avg_CE = CE.mean(axis=0)   




/var/folders/k0/5kjb5xhn5hz5qd2_5ccp61th0000gn/T/ipykernel_50157/1618677704.py:37: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



In [98]:
avg_CE=np.insert(avg_CE, 18, [0,0])

In [99]:
avg_CE

array([0.74060817, 0.84390585, 0.71434743, 0.93280269, 0.51336907,
       0.68045722, 0.91024507, 0.93059912, 0.68081367, 0.77351469,
       0.78601841, 0.55514533, 0.06621536, 0.06628096, 1.        ,
       0.64168626, 0.73486231, 0.59175174, 0.        , 0.        ,
       0.39016367, 0.37999285, 0.43272257, 0.18915197, 0.32418419,
       0.4220337 , 0.59827717])

In [100]:
df["Cross_Efficiency_Avg_1500"] = avg_CE

In [101]:
df[["Model", "Cross_Efficiency_Avg_300", "Cross_Efficiency_Avg_1000", "Cross_Efficiency_Avg_1500"]]

,Model,Cross_Efficiency_Avg_300,Cross_Efficiency_Avg_1000,Cross_Efficiency_Avg_1500
0,GPT-4.1,0.643694,0.685608,0.740608
1,GPT-4.1 mini,0.740521,0.764052,0.843906
2,GPT-4.1 nano,0.719568,0.725861,0.714347
3,o4-mini (high),0.715453,0.941840,0.932803
4,GPT-4.5,0.418056,0.393562,0.513369
5,o3,0.607045,0.495107,0.680457
6,o3-mini (high),0.698657,0.531050,0.910245
7,o3-mini,0.805018,0.749024,0.930599
8,o1,0.571803,0.598059,0.680814
9,o1-mini,0.706002,0.515956,0.773515


In [102]:
df["Cross_Efficiency_Avg_Total"] = (
    df[["Cross_Efficiency_Avg_300", "Cross_Efficiency_Avg_1000", "Cross_Efficiency_Avg_1500"]]
    .replace(0, pd.NA)  
    .mean(axis=1, skipna=True)  
)


In [103]:
df[["Model", "Cross_Efficiency_Avg_Total"]]

,Model,Cross_Efficiency_Avg_Total
0,GPT-4.1,0.68997
1,GPT-4.1 mini,0.782826
2,GPT-4.1 nano,0.719926
3,o4-mini (high),0.863365
4,GPT-4.5,0.441662
5,o3,0.594203
6,o3-mini (high),0.713317
7,o3-mini,0.828214
8,o1,0.616892
9,o1-mini,0.665157


In [104]:


input_cols = [
    "Critical Power (kW)",
    "Mean Energy Consumption (1000 Output Tokens)",
        "Mean Energy Consumption (300 Output Tokens)",

    "PUE",
    "WUE (Site)",
    "WUE (Source)",
    "CIF"
]
output_cols = ["Cross_Efficiency_Avg_Total", 'Score']
all_cols = input_cols + output_cols

df = df.copy()
X = df[all_cols].values
X_scaled = StandardScaler().fit_transform(X)

pca = PCA(n_components=2)
pcs = pca.fit_transform(X_scaled)

pca_df = pd.DataFrame(pcs, columns=['PC1','PC2'], index=df.index)
pca_df['Model'] = df['Model']
pca_df['Cross_Efficiency'] = df['Cross_Efficiency_Avg_Total']

fig_scree = px.bar(
    x=['PC1','PC2'], y=pca.explained_variance_ratio_,
    labels={'x':'Principal Component','y':'Variance Ratio'},
    title='Explained Variance by PC'
)
fig_scree.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig_scree.show()

scale = max(pca_df[['PC1','PC2']].abs().max()) * 0.7
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=pca_df['PC1'], y=pca_df['PC2'],
    mode='markers+text',
    text=pca_df['Model'], textposition='top center',
    marker=dict(size=8, color=pca_df['Cross_Efficiency'], colorscale='Viridis',
                colorbar=dict(title='Cross-Eff'))
))

for i, var in enumerate(all_cols):
    x_arrow = pca.components_[0, i] * scale
    y_arrow = pca.components_[1, i] * scale

    fig.add_annotation(
        x=x_arrow, y=y_arrow,
        ax=0, ay=0,
        xref='x', yref='y', axref='x', ayref='y',
        showarrow=True, arrowhead=3, arrowwidth=2, arrowcolor='red'
    )
    fig.add_annotation(
        x=x_arrow*1.05, y=y_arrow*1.05,
        text=var, showarrow=False, font=dict(color='red')
    )

fig.update_layout(
    title='PCA Biplot (Inputs + Cross-Eff)',
    xaxis_title='PC1', yaxis_title='PC2',
    plot_bgcolor='white', paper_bgcolor='white'
)
fig.show()



In [105]:
input_cols = [
    "Critical Power (kW)",
    "Mean Energy Consumption (1000 Output Tokens)",
        "Mean Energy Consumption (300 Output Tokens)",

    "PUE",
    "WUE (Site)",
    "WUE (Source)",
    "CIF"
]
output_cols = ["Cross_Efficiency_Avg_Total", 'Score']
all_cols = input_cols 

df = df.copy()
X = df[all_cols].values
X_scaled = StandardScaler().fit_transform(X)

pca = PCA(n_components=7)
pcs = pca.fit_transform(X_scaled)

pca_df = pd.DataFrame(pcs, columns=['PC1','PC2',"PC3","PC4", 'PC5','PC6','PC7'], index=df.index)
pca_df['Model'] = df['Model']
pca_df['Cross_Efficiency'] = df['Cross_Efficiency_Avg_Total']

fig_scree = px.bar(
    x=['PC1','PC2',"PC3","PC4", 'PC5','PC6','PC7'], y=pca.explained_variance_ratio_,
    labels={'x':'Principal Component','y':'Variance Ratio'},
    title='Explained Variance by PC'
)
fig_scree.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig_scree.show()

scale = max(pca_df[['PC1','PC2']].abs().max()) * 0.7
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=pca_df['PC1'], y=pca_df['PC2'],
    mode='markers+text',
    text=pca_df['Model'], textposition='top center',
    marker=dict(size=8, color=pca_df['Cross_Efficiency'], colorscale='Viridis',
                colorbar=dict(title='Cross-Eff'))
))

for i, var in enumerate(all_cols):
    x_arrow = pca.components_[0, i] * scale
    y_arrow = pca.components_[1, i] * scale

    fig.add_annotation(
        x=x_arrow, y=y_arrow,
        ax=0, ay=0,
        xref='x', yref='y', axref='x', ayref='y',
        showarrow=True, arrowhead=3, arrowwidth=2, arrowcolor='red'
    )
    fig.add_annotation(
        x=x_arrow*1.05, y=y_arrow*1.05,
        text=var, showarrow=False, font=dict(color='red')
    )

fig.update_layout(
    title='PCA Biplot (Inputs + Cross-Eff)',
    xaxis_title='PC1', yaxis_title='PC2',
    plot_bgcolor='white', paper_bgcolor='white'
)
fig.show()



In [107]:
df.loc[df["Model"].isin(["GPT-4o mini","GPT-4o (Mar '25)"])].iloc[:,19:25]

,Min Mean (300 Output Tokens),Max Mean (300 Output Tokens),Min Mean (1k Output Tokens),Max Mean (1k Output Tokens),Min Mean (1.5k Output Tokens),Max Mean (1.5k Output Tokens)
10,0.000386,0.000456,0.001112,0.001316,0.001638,0.001938
11,0.000405,0.000437,0.001364,0.001471,0.002027,0.002185


In [108]:
df.loc[df["Model"]=="Claude 3.5 Haiku", "Model"]= "Claude-3.5 Haiku"

In [109]:
df_melted=df[["Model", "Cross_Efficiency_Avg_300", "Cross_Efficiency_Avg_1000", "Cross_Efficiency_Avg_1500"]].melt(id_vars=["Model"], var_name="Prompt Length", value_name="Cross-Efficiency")

In [111]:
pastel_colors = {
    'OpenAI':    '#3FA98C',
    'Anthropic': '#D3B641',
    'Deepseek':  '#4B8DB4',
    'Meta':      '#9D88AF'
}
df_sorted = df.sort_values(by='Cross_Efficiency_Avg_Total', ascending=False)
model_order = df_sorted['Model'].unique()
df['label'] = df['Score'].astype(str).apply(lambda x: x.split('.')[0]) + "<br>" + df['Cross_Efficiency_Avg_Total'].apply(lambda x: f"{x:.3f}")
fig = px.bar(
    df,
    x='Model',
    y='Cross_Efficiency_Avg_Total',
    color='Organization',
    color_discrete_map=pastel_colors,
    text='label',
    title='DEA Cross Efficiency Score by Model',
    category_orders={'Model': model_order}
)

fig.update_traces(
    textposition='outside'
)

fig.update_yaxes(
    title='Cross Efficiency Score',
    tickfont=dict(size=20),

    showgrid=True,
    gridcolor='lightgray',
    showline=True,
    linecolor='black',
    ticks='outside',
    ticklen=6,
    tickwidth=1,
    tickcolor='black',
    dtick=0.1,
    range=[0,1]
)
fig.update_xaxes(
    title='Model',
    tickangle=45,
    tickfont=dict(size=20),
    showgrid=False,
    showline=True,
    linecolor='black',
    ticks='outside',
    ticklen=6,
    tickwidth=1,
    tickcolor='black'
)

fig.update_layout(
    height=1000,
    width=2500,
    title_x=0.4,
    plot_bgcolor='white',
    bargap=0.45,
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    font=dict(family='Arial', size=20, color='black'),
    legend_title_text='Company',
    legend=dict(
        title_text='Company',
        orientation='v',
        yanchor='top',
        y=0.92,
        xanchor='right',
        x=1,
        bordercolor='black',
        borderwidth=1,
        bgcolor='rgba(255,255,255,0.8)',
        title_font=dict(size=25, family='Arial', color='black'),
        font=dict(size=20, family='Arial', color='black'),
        itemclick='toggle',
        itemdoubleclick='toggleothers'
    ),
    margin=dict(t=80, b=80, r=80)
)

fig.show()

In [114]:
df[["Model", "Score", "Cross_Efficiency_Avg_Total"]].to_csv('DEATotal.csv')